In [1]:
import numpy as np
import pandas as pd

import pyodbc
import re, collections
import string

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import pyLDAvis.gensim
from gensim.test.utils import datapath

from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
import nltk.data

from textblob import TextBlob

import seaborn as sns
import matplotlib.pyplot as plt
import datetime

from configparser import ConfigParser

import warnings
warnings.filterwarnings("ignore")

/Users/hakanpolat/.local/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
/Users/hakanpolat/.local/lib/python3.7/site-packages/nltk/lm/counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, defaultdict


In [64]:
# df = pd.read_csv(myfile,sep='\t',skiprows=(0,1,2),header=(0))
datat = pd.read_csv('./Input/CA.txt',encoding=" ISO-8859-1", sep='\t',header=(0))

datac = pd.read_csv('./Input/CA.csv',encoding=" ISO-8859-1", sep='\t',header=(0))

datax = pd.read_excel('./Input/CA.xlsx')    

In [65]:
import timeit

In [66]:
datat.memory_usage()

Index          80
climsg      33792
globalid    33792
dtype: int64

In [68]:
%timeit datat

88 ns ± 15.6 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [69]:
datac.memory_usage()

Index                 80
climsg,globalid    33792
dtype: int64

In [70]:

%timeit datac.memory_usage()

2.24 ms ± 132 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [71]:
datax.memory_usage()

Index          80
climsg      33792
globalid    33792
dtype: int64

In [72]:
%timeit datax.memory_usage()

2.49 ms ± 317 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [40]:

def pyAsterquery(text, maxlimit):
    conn = pyodbc.connect('DRIVER={Aster ODBC Driver};SERVER =tdwtddpast27-2.tdc.vzwcorp.com;PORT = 2406;DATABASE=vzwdisc02;UID=mylavra;PWD=welcome02')
    data = pd.read_sql(text + ' limit ' + str(maxlimit),conn)
    return data

def pySQL(schm, tbl, columns, cndtns):
    return "select " + columns + " from " + schm + "." + tbl + " where " + cndtns

def pyreadcsv(iname):
    data = pd.read_csv('./Input/'+iname+'.csv')
    return data

def pyreadtxt(iname):
    data = pd.read_csv('./Input/'+iname+'.txt',encoding=" ISO-8859-1", sep='\t',header=(0) )

def pyreadxl(iname):
    data = pd.read_excel('./Input/'+iname+'.xlsx')
    return data

def pyremovenulls(df,msg):
    df = df[df[msg].isnull()==False]
    df.reset_index(drop=True,inplace=True)
    return df

def applyfilters(colname, msgsearch, interacn):
    interacn = ' ' + interacn + ' '
    msgsearch = msgsearch.lower()
    msgsearch = list(msgsearch.split(','))
    for i in range(len(msgsearch)):
        msgsearch[i] = colname + ' ilike ' + "'" +'%' + msgsearch[i] + '%' + "'"
    return '(' + interacn.join(msgsearch) + ')'


def pymsgsearchSQL(schm, tbl, columns, cndtns, colname, msgsearch,interacn):
    return pySQL(schm, tbl, columns, cndtns) + ' and ' + applyfilters(colname, msgsearch,interacn)

def pymsgSearchfile(df,msg,text,intrcn):
    text = text.lower()
    text = list(text.split(','))
    if intrcn == 'and':
        df['contains'] = df[msg].apply(lambda x: all(word in x.lower() for word in text))
    else:
        df['contains'] = df[msg].apply(lambda x: any(word in x.lower() for word in text))
    df = df[df['contains'] == True]
    df.drop(['contains'],axis = 1, inplace = True)
    df.reset_index(drop=True,inplace=True)
    return df

def pynametopics(df,msg,ntopics,npasses,topic_length):
    stemmer = SnowballStemmer('english')
    user_stopwords = pd.read_csv('./Input/Stopwords.txt',sep=',',encoding = "ISO-8859-1",header = 0)
    def lemmatize_stemming(text):
        return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
    def preprocess(text):
        result = [] 
        for token in gensim.utils.simple_preprocess(text):
            if (token not in STOPWORDS and token not in np.array(user_stopwords['Stopwords']) and len(token) > 3
                and lemmatize_stemming(token) not in STOPWORDS and lemmatize_stemming(token) not in np.array(user_stopwords['Stopwords'])):
                result.append(lemmatize_stemming(token))
        return result
    def run_lda(df,msg,ntopics,npasses):
        chats_preprocessed = df[msg].apply(preprocess)
        dictionary = gensim.corpora.Dictionary(chats_preprocessed)
        dictionary.filter_extremes(no_below=10, no_above=0.5)
        bow_corpus = [dictionary.doc2bow(doc) for doc in chats_preprocessed]
        lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=ntopics, id2word=dictionary, passes=npasses)
        topics = []
        for i in range(len(bow_corpus)):
            x = dict(lda_model.get_document_topics(bow_corpus[i]))
            topics.append(max(x.keys(), key=(lambda key: x[key])))
        df['topics'] = topics
        return df, dictionary, bow_corpus, lda_model
    df,dictionary,bow,lda = run_lda(df,msg,ntopics,npasses)
    def get_topics(df,lda_model,ntopics,topic_length):
        topic_words = []
        for i in range(ntopics):
            x = lda_model.print_topic(i,topn=7)
            topic_words.append(re.sub('[^a-zA-Z]+',' ',x).split(' ')[1:-1])
        d = {}
        for j in range(7):
            for i in range(ntopics):
                if topic_words[i][j] not in d:
                    d[topic_words[i][j]] = i
        a = np.array(list(d.values()))
        b = np.array(list(d.keys()))
        topics_name = []
        for i in range(ntopics):
            topics_name.append(' & '.join(list(b[np.where(a == i)])[:topic_length]))
        df['topics'] = df['topics'].apply(lambda x: topics_name[x])
        df['topics'] = df['topics'].apply(lambda x: x.upper())
        return df, topic_words, topics_name
    df, topic_words, topic_names = get_topics(df,lda,ntopics,topic_length)
    # Save model to disk.
    # temp_file = datapath("model")
    # lda.save(temp_file)

    # Load a potentially pretrained model from disk.
    # lda = LdaModel.load(temp_file)
    return df,dictionary,bow,lda,topic_words,topic_names
def pySentiment(df,msg):
    def pyScore(score):
        if score > 0.1:
            return 'Positive'
        elif score < -0.1:
            return 'Negative'
        else:
            return 'Neutral'
    df['Score'] = df[msg].apply(lambda x: TextBlob(x).sentiment.polarity)
    df['Sentiment'] = df['Score'].apply(lambda x:pyScore(x))
    df.drop(['Score'],axis=1,inplace=True)
    return df

def pygetcurrenttime():
    x = str(datetime.datetime.now()).replace(' ','_')
    x = x.replace(':','')
    x = x.replace('.','')
    return x

def pysavedata(df,typ,x):
    if typ == 'csv':
        df.to_csv('./Output/Data/'+x+'.csv',sep = ',',index=False)
    elif typ == 'xlsx':
        df.to_excel('./Output/Data/'+x+'.xlsx',index=False)

def pyLDAvissave(lda,bow,dictionary,x):
    p = pyLDAvis.gensim.prepare(lda, bow, dictionary,sort_topics = True)
    pyLDAvis.save_html(p,'./Output/pyLDAVis/'+x+'.html')

def pytopicsbarvis(df,x):
    sns.set(rc={'figure.figsize':(25,8.27)})
    sns.set(style="darkgrid")
    sns.set_context("paper", font_scale=1.5)
    total = float(len(df)) # one person per row
    ax = sns.countplot(x="topics", data=df,order = df['topics'].value_counts().index)
    ax.set_title('Topic Frequency',fontsize=20)
    ax.set_xlabel('Top Topics',fontsize=16)
    ax.set_ylabel('Percentage(%)',fontsize=16)
    ax.legend(loc='upper left',fontsize=20)
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.3f}'.format(int(height/total*100)),
            ha="center")
    # if you do not want to display it, and just to save it, use below code
    plt.tight_layout()
    plt.close()
    ax.get_figure().savefig('./Output/Plots/frequency/Frequency_'+x+'.png')

def pysentbarvis(df,x):
    sns.set(rc={'figure.figsize':(28,8.27)})
    sns.set(style="darkgrid")
    sns.set_context("paper", font_scale=1.5)
    total = float(len(df)) # one person per row
    ax = sns.countplot(x="topics", hue="Sentiment", data=df,order = df['topics'].value_counts().index)
    ax.set_title('Sentiment Analysis',fontsize=20)
    ax.set_xlabel('TOPICS',fontsize=20)
    ax.set_ylabel('Percentage(%)',fontsize=20)
    ax.legend(loc='upper left',fontsize=20,labels=['Neutral','Positive','Negative'])
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.3f}'.format(height/total*100),
            ha="center")
    # if you do not want to display it, and just to save it, use below code
    plt.tight_layout()
    plt.close()
    ax.get_figure().savefig('./Output/Plots/sentiment/Sentiment_'+x+'.png')


In [14]:
parser = ConfigParser()
parser.read('LDA_Config.ini')

input_typ = parser.get('Input_Type','input_typ')
colname = parser.get('colname','colname')
if input_typ == 'Custom_SQL':
    df = pyAsterquery(parser.get('Custom_SQL','custom_sql'),parser.get('Max_Rows_SQL','max_limit'))
elif input_typ == 'Predefined_SQL':
    schema_name = parser.get('Predefined_SQL','schema_name')
    table_name = parser.get('Predefined_SQL','table_name')
    columns = parser.get('Predefined_SQL','columns')
    conditions = parser.get('Predefined_SQL','conditions')
    filter_data = parser.get('Predefined_SQL','filter_data')
    max_rows = parser.get('Max_Rows_SQL','max_limit')
    if filter_data == True:
        filter_values = parser.get('Predefined_SQL','filter_values')
        interaction = parser.get('Predefined_SQL','interaction')
        sql_query = pymsgsearchSQL(schema_name, table_name, columns, conditions, colname, filter_values,interaction)
        df = pyAsterquery(sql_query,max_rows)
    else:
        sql_query = pySQL(schema_name, table_name, columns, conditions)
        df = pyAsterquery(sql_query,max_rows)
elif input_typ == 'csv':
    filename = parser.get('csv','file_name')
    df = pyreadcsv(filename)
    filter_data = parser.get('csv','filter_data')
    if filter_data == True:
        filter_values = parser.get('csv','filter_values')
        interaction = parser.get('csv','interaction')
        df = pymsgSearchfile(df,colname,filter_values,interaction)
elif input_typ == 'txt':
    filename = parser.get('txt','file_name')
    df = pd.read_table(filename)
    filter_data = filter_data = parser.get('txt','filter_data')
    if filter_data == True:
        filter_values = parser.get('txt','filter_values')
        interaction = parser.get('txt','interaction')
        df = pymsgSearchfile(df,colname,filter_values,interaction)
    
elif input_typ == 'xlsx':
    filename = parser.get('xlsx','file_name')
    df = pyreadxl(filename)
    filter_data = parser.get('xlsx','filter_data')
    if filter_data == True:
        filter_values = parser.get('xlsx','filter_values')
        interaction = parser.get('xlsx','interaction')
        df = pymsgSearchfile(df,colname,filter_values,interaction)



FileNotFoundError: [Errno 2] File b'dprelated2' does not exist: b'dprelated2'

In [7]:
if __name__ == '__main__':
    df['climsg'] = df['climsg'].apply(str)
    df = pyremovenulls(df,colname)
#     df = df.dropna(subset=['msg'])
    num_topics = parser.get('LDA_Topic_Modeling','num_topics')
    num_passes = parser.get('LDA_Topic_Modeling','num_passes')
    topic_length = parser.get('Topic_Length','topic_length')
    df,dictionary,bow,lda,topic_words,topic_names = pynametopics(df,colname,int(num_topics),int(num_passes),int(topic_length))
    df = pySentiment(df,colname)
    current_date_time = pygetcurrenttime()
    pysavedata(df,parser.get('save_data','type'),current_date_time)
    pyLDAvissave(lda,bow,dictionary,current_date_time)
    pytopicsbarvis(df,current_date_time)
    pysentbarvis(df,current_date_time)


No handles with labels found to put in legend.


In [8]:
topic_words

[['credit', 'watch', 'order', 'check', 'payment', 'transfer', 'applic'],
 ['charg', 'pay', 'dollar', 'switch', 'plan', 'cost', 'wonder'],
 ['plan', 'discount', 'unlimit', 'data', 'switch', 'price', 'offer'],
 ['store', 'order', 'pick', 'ship', 'case', 'protect', 'promo'],
 ['trade', 'cart', 'order', 'account', 'credit', 'balanc', 'pay'],
 ['payment', 'price', 'watch', 'plan', 'contract', 'pay', 'deal'],
 ['order', 'account', 'address', 'ship', 'space', 'payment', 'confirm']]

In [9]:
topic_names

['credit & watch',
 'charg & pay',
 'plan & discount',
 'store & pick',
 'trade & cart',
 'payment & price',
 'order & account']

# N-Gram

In [10]:
lda.show_topics()

[(0,
  '0.113*"credit" + 0.075*"watch" + 0.045*"order" + 0.034*"check" + 0.029*"payment" + 0.014*"transfer" + 0.014*"applic" + 0.013*"ask" + 0.013*"trade" + 0.012*"save"'),
 (1,
  '0.066*"charg" + 0.056*"pay" + 0.044*"dollar" + 0.042*"switch" + 0.042*"plan" + 0.027*"cost" + 0.027*"wonder" + 0.026*"tax" + 0.025*"account" + 0.025*"fee"'),
 (2,
  '0.170*"plan" + 0.084*"discount" + 0.060*"unlimit" + 0.032*"data" + 0.030*"switch" + 0.018*"price" + 0.017*"offer" + 0.016*"atampampt" + 0.014*"share" + 0.014*"deal"'),
 (3,
  '0.124*"store" + 0.108*"order" + 0.059*"pick" + 0.028*"ship" + 0.027*"case" + 0.022*"protect" + 0.017*"promo" + 0.015*"date" + 0.014*"deliv" + 0.014*"deliveri"'),
 (4,
  '0.176*"trade" + 0.063*"cart" + 0.042*"order" + 0.034*"account" + 0.024*"credit" + 0.022*"balanc" + 0.019*"pay" + 0.017*"checkout" + 0.014*"return" + 0.014*"tradein"'),
 (5,
  '0.096*"payment" + 0.047*"price" + 0.037*"watch" + 0.034*"plan" + 0.028*"contract" + 0.028*"pay" + 0.026*"deal" + 0.025*"add" + 0.02

In [35]:
import nltk, re, string, collections
from nltk.util import ngrams


def lemmatize_stemming(text):
        return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if (token not in STOPWORDS and token not in np.array(user_stopwords['Stopwords']) and len(token) > 3 
            and lemmatize_stemming(token) not in STOPWORDS and lemmatize_stemming(token) not in np.array(user_stopwords['Stopwords'])):
            result.append(lemmatize_stemming(token))
    return result


stemmer = SnowballStemmer('english')
user_stopwords = pd.read_csv('./Input/Stopwords.txt',sep=',',encoding = "ISO-8859-1",header = 0)

In [37]:
cpc = pd.read_excel('cpc.xlsx')

In [49]:
len(cpc['climsg'])

3016

In [50]:
text = ''
for i in cpc['climsg']:
    text = text + str(i)
    
text = preprocess(text)
#get a list of all the n-grams
esBigrams = ngrams(text, 3)
# get the frequency of each bigram in our corpus
esBigramFreq = collections.Counter(esBigrams)
# what are the ten most popular ngrams?
esBigramFreq.most_common(40)

[(('plan', 'unlimit', 'data'), 6),
 (('talk', 'real', 'person'), 6),
 (('unlimit', 'data', 'plan'), 5),
 (('offer', 'unlimit', 'plan'), 4),
 (('best', 'deal', 'deal'), 2),
 (('inform', 'intern', 'plan'), 2),
 (('inform', 'price', 'price'), 2),
 (('deal', 'requir', 'deal'), 2),
 (('deal', 'incent', 'deal'), 2),
 (('data', 'roll', 'data'), 2),
 (('expir', 'expir', 'date'), 2),
 (('reward', 'expir', 'expir'), 2),
 (('day', 'data', 'leav'), 2),
 (('unlimit', 'talk', 'data'), 2),
 (('data', 'plan', 'unlimit'), 2),
 (('claim', 'insur', 'claim'), 2),
 (('respons', 'interfac', 'excel'), 2),
 (('interfac', 'excel', 'account'), 2),
 (('excel', 'account', 'inform'), 2),
 (('account', 'inform', 'clear'), 2),
 (('inform', 'clear', 'excel'), 2),
 (('clear', 'excel', 'construct'), 2),
 (('excel', 'construct', 'improv'), 2),
 (('construct', 'improv', 'interact'), 2),
 (('cost', 'plan', 'pay'), 2),
 (('loyal', 'deal', 'incent'), 2),
 (('offer', 'requir', 'smartphon'), 2),
 (('guess', 'switch', 'deal'),